In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from utils import calculate_mse
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/test.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

N = tX.shape[0]
# add column of ones to add bias term
tx = np.hstack((np.ones((N, 1)), tX))

D = tX.shape[1]

In [ ]:
# use only outlier-free data for plotting
tX_plot = np.array([row for row in tX if -999. not in row])
tX_plot.shape

In [ ]:
# NAs are marked as -999
print("tX.head = ", tX[:5,:5])
#print(tX[:5,5:10])
#print(tX[:5,10:15])
print("tX.shape = ",tX.shape)
print("tX.size = ", tX.size, "\n")

print("y.head = ",y[:10])

print(np.where(y == -1))
print(np.unique(y))
print("y.shape = ",y.shape)
print("y.size = ", y.size, "\n")

print("ids.head = ",ids[:10])
print("ids.shape = ", ids.shape)
print("ids.size = ", ids.size, "\n")


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

ax1.boxplot(tX[:100,:15]);
ax2.boxplot(tX[:100,15:]);

In [ ]:
fig, ax = plt.subplots()

ax.hist(tX_plot[:100,:1], bins = 100, histtype='step', stacked=True, fill=False);

# Do your crazy machine learning thing here :) ...

## 1. Exploratory data analysis

## 2. Preprocessing

In [ ]:
# Normalize dataset
for indx_col in range(tX.shape[1]):
    tX[:, indx_col] = (tX[:, indx_col] - np.mean(tX[:, indx_col]))/np.std(tX[:, indx_col])

print(y)
# Split dataset
split_perc = 0.7
split_ind = int(len(y) * split_perc)

# Training set
y_train = y[:split_ind]
tX_train = tX[:split_ind]
ids_train = ids[:split_ind]

# Validation set
y_val = y[split_ind:]
tX_val = tX[split_ind:]
ids_val = ids[split_ind:]

## 3. Training

In [ ]:
# Set hyperparameters
max_iters = 100
gamma = 0.01
lambda_ = 0.1

# Initialize weights
initial_w = np.random.rand(D,)

from implementations import *
# Train
#weights_ls_GD, loss_ls_GD = least_squares_GD(y_train, tX_train, initial_w, max_iters, gamma)
#print(loss_ls_GD)
#weights_ls_SGD, loss_ls_SGD = least_squares_SGD(y_train, tX_train, initial_w, max_iters, gamma)
#weights_ls, loss_ls = least_squares(y_train, tX_train)
# weights_rr, loss_rr = ridge_regression(y_train, tX_train, lambda_)
# weights_lr, loss_lr = logistic_regression(y_train, tX_train, initial_w, max_iters, gamma)
# weights_reg_lr, loss_reg_lr = reg_logistic_regression(y_train, tX_train, lambda_, initial_w, max_iters, gamma)

## 4. Evaluation

In [ ]:
from utils import cross_validation, build_k_indices
# Cross-validation

k = 5
k_indices = build_k_indices(y, k)
ls_GD_losses = []
ls_losses = []

for k_iteration in range(k):
    tX_train, y_train, tX_val, y_val = cross_validation(y, tX, k_indices, k_iteration)
    weights_ls_GD, loss_ls_GD = least_squares_GD(y_train, tX_train, initial_w, max_iters, gamma)
    weights_ls, loss_ls = least_squares(y_train, tX_train)

    y_pred_ls_GD_val = tX_val @ weights_ls_GD
    y_pred_ls_val = tX_val @ weights_ls

    ls_GD_losses.append(calculate_mse_loss(y_val, y_pred_ls_val))
    ls_losses.append(calculate_mse_loss(y_val, y_pred_ls_val))

ls_GD_losses = np.array(ls_GD_losses)
ls_losses = np.array(ls_losses)

ls_GD_mean_loss = np.mean(ls_GD_losses)
ls_mean_loss = np.mean(ls_losses)

print(ls_GD_mean_loss, ls_mean_loss)

## Generate predictions and save output in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
# TODO: decide real weights
weights = least_squares_GD(y, tX, initial_w, max_iters, gamma)
OUTPUT_PATH = '../data/submission.csv'
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)